# Preprocessing

In [20]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion VAD

In [16]:
def calculate_vad(audio_signal, sample_rate=16000, frame_length=1024, hop_length=512, threshold=0.05):
    """
    Berechnet den Prozentsatz der Sprachaktivität (VAD) in einem Audiosignal.

    Quelle:
        Maël Fabien, "Voice Activity Detection", 
        https://maelfabien.github.io/machinelearning/Speech4/#
        
    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).
        frame_length (int): Länge eines Frames in Samples (Standard: 1024).
        hop_length (int): Schrittweite zwischen Frames in Samples (Standard: 512).
        threshold (float): Schwellenwert für die Energie, um Sprachaktivität zu erkennen.

    Returns:
        float: Prozentsatz der Sprachaktivität im Signal.
    """
    # Energie pro Frame berechnen
    energy = librosa.feature.rms(y=audio_signal, frame_length=frame_length, hop_length=hop_length)[0]

    # Sprachaktive Frames zählen (Energie > Threshold)
    num_active_frames = np.sum(energy > threshold)

    # Gesamtanzahl der Frames
    total_frames = len(energy)

    # Prozentsatz der Sprachaktivität
    vad_percentage = (num_active_frames / total_frames) * 100
    
    return vad_percentage

# Test

In [17]:
import os

# VAD für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)
        
        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)
        
        # VAD berechnen
        vad_percentage = calculate_vad(audio_signal)
        
        print(f"Voice Activity Detection ({file_name}): {vad_percentage:.2f}%")

Voice Activity Detection (_noise_pink.wav): 100.00%
Voice Activity Detection (_noise_white.wav): 100.00%
Voice Activity Detection (_signal_constant.wav): 100.00%
Voice Activity Detection (_signal_silence.wav): 0.00%
Voice Activity Detection (_signal_sine.wav): 100.00%
Voice Activity Detection (_snr_03.wav): 86.92%
Voice Activity Detection (_snr_10.wav): 66.36%
Voice Activity Detection (_snr_20.wav): 60.75%
Voice Activity Detection (example1.wav): 51.66%
Voice Activity Detection (example2.wav): 57.84%
Voice Activity Detection (example3.wav): 42.86%
